# RNN 예제

TensorFlow 2.0을 통해 자연어 데이터를 전처리하고, RNN을 사용하는 예제를 살펴봅니다.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

## IMDB 데이터셋 로드

IMDB 데이터셋을 로드합니다.

IMDB 데이터셋은 영어로 작성된 영화에 대한 리뷰를 모아놓은 데이터셋으로,

리뷰의 내용에 따라 긍정적인 경우 1로, 부정적인 경우 0으로 label되어있습니다.

In [ ]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data, test_data = imdb['train'], imdb['test']

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

# 텍스트는 텍스트끼리, label은 label끼리 따로 분류
for s, l in train_data:

  # training_sentences.append(s.numpy().decode('utf8'))
  
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())
  
for s, l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())
  
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## 로드한 데이터 확인

예시로 두 개의 데이터만 확인해보겠습니다.

출력된 두 리뷰를 보면 첫 번째 리뷰는 부정적이고, 0으로 label되어있습니다.

두 번째 리뷰는 긍정적이고, 1으로 label되어있습니다.

In [ ]:
for i in [0, 14]:
  print(train_labels[i], train_sentences[i])

## 데이터 전처리

이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

전처리에 사용할 함수들을 import하고, 각종 parameter를 설정합니다.

그 후, 해당 parameter들을 바탕으로 전처리를 진행합니다.

데이터 전처리가 어떻게 진행되는지 확인하기 위해 0번 데이터와 2번 데이터의 변화사항을 출력합니다.

### Tokenizing
1. Tokenizer instantiate
2. fit_on_texts
3. work_index : word index를 갖는 dictionary 생성
4. texts_to_sequences : 각 문장의 벡터값들을 하나의 sequence로 합치기
5. pad_sequences : 문장마다 길이가 다르므로 길이를 맞춰주기 위해서 0을 채움.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000 # 단어사전의 크기
embedding_dim = 16 # 토큰 임베딩 차원 수
max_length = 120 # 입력 시퀀스의 최대 길이를 지정
trunc_type='post' # max_length보다 긴 시퀀스의 경우 어디를 잘라낼 것인지 지정. pre는 앞쪽을, post는 뒷쪽을 잘라냄
oov_tok = "<OOV>" # Out Of Vocabulary 토큰
padding_type='post'

print("원본 데이터 0번:", train_sentences[0])
print("원본 데이터 2번:", train_sentences[2])
print("#" * 30)

# 크기가 vocab_size인 단어사전을 만듦
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

# 위에서 만든 단어사전을 바탕으로 단어들을 정수 인덱스로 변환
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

print("정수로 인코딩 후 0번:", train_sequences[0])
print("길이:", len(train_sequences[0]))
print("정수로 인코딩 후 2번:", train_sequences[2])
print("길이:", len(train_sequences[2]))
print("#" * 30)

# pad_sequences 함수를 통해 padding 혹은 truncating하여 모든 데이터의 길이를 max_length로 통일
# 길이가 짧은 데이터는 padding을 추가하고, 길이가 긴 데이터는 trunc_type 설정에 따라 잘라냄

#  padding=padding_type 으로 옵션 주기도 한다.

train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

print("Padding/Truncating 후 0번:", train_padded[0])
print("길이:", len(train_padded[0]))
print("Padding/Truncating 후 2번:", train_padded[2])
print("길이:", len(train_padded[2]))

출력된 결과를 보면, 영어로 된 원본 텍스트 데이터가

토크나이징 & 인코딩 후에는 정수의 list로 변환됩니다.

길이는 각각 118, 133입니다.

<br>

Padding/Truncating 후에는 길이가 120으로 통일되었습니다.

길이가 118이던 0번 데이터는 padding으로 인해 앞에 0 두개가 추가되었습니다.

길이가 133이던 2번 데이터는 truncating으로 인해 뒤쪽 13개가 삭제되었습니다.

## 네트워크 정의

tf.keras.models.Sequential을 이용해 RNN을 정의합니다.

먼저, 임베딩 레이어가 필요합니다.

임베딩 레이어는 단어사전 크기, 임베딩 크기, 시퀀스 길이를 argument로 받습니다.

<br>

임베딩 후에 RNN Layer를 추가합니다.

RNN Layer는 SimpleRNN, LSTM, GRU 등이 있습니다.

<br>

그 후 Dense Layer를 추가합니다.

일반적으로 RNN Layer 이후에 Dense Layer 하나를 먼저 추가하고,

task에 맞는 node 개수를 가진 Output Dense Layer를 추가합니다.

이진 분류 문제이므로 Output Layer에서는 sigmoid를 사용합니다.

<br>

이진 분류 문제이므로 binary_crossentropy를 loss로 사용하여 모델을 컴파일합니다.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

'''
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), #임베딩 레이어 추가
    tf.keras.layers.GlobalAveragePooling1D(), #Flatten보다 GlobalAveragePooling1D 사용하기
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
'''

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## 학습

학습 후 결과를 확인합니다.

In [ ]:
num_epochs = 10
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels))

## 테스트

직접 만든 테스트용 문구로 모델을 테스트합니다.

첫 번째 문구에 대해서는 긍정인 1에 가까운 값을, 두 번째 문구에 대해서는 부정인 0에 가까운 값을 보입니다.

In [ ]:
my_sentences = ["I cannot describe how great is this film in this narrow space.",
                "Who is the director? Even I can make better than this."]

my_sequences = tokenizer.texts_to_sequences(my_sentences)
my_padded = pad_sequences(my_sequences, maxlen=max_length, truncating=trunc_type)

model.predict(my_padded)